# Multi-Class Classification with Machine Learning
In this notebook, we will explore various machine learning models to solve a multi-class classification problem. We will evaluate and compare the performance of different algorithms on the dataset.


In [1]:
import ast
import random
from collections import Counter
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.model_selection import iterative_train_test_split
from tqdm import tqdm

from preprocess_functions import build_tree, extract_keys, preprocess_texts
from utils import CalibratedLabelRankClassifier, ChainOfClassifiers, LabelPowersetClassifier, \
    assess_models, prune_and_subsample, ConditionalDependencyNetwork, MetaBinaryRelevance


In [2]:
OVERWRITE = False
RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)


In [3]:
INIT_POINTS = 1
N_ITER = 5
TEST_SIZE = 2e-1

BASE_CLASSIFIERS = {
    'logistic_regression': LogisticRegression(solver='liblinear', random_state=RANDOM_STATE),
    'gaussian_nb': GaussianNB(),
    'decision_tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'random_forest': RandomForestClassifier(random_state=RANDOM_STATE),
    'xgb': xgb.XGBClassifier(random_state=RANDOM_STATE),
}

COLAB_PATH = Path('/content/drive/MyDrive')
KAGGLE_PATH = Path('/kaggle/input')
LOCAL_PATH = Path('./')

# Step 1: Check if running in Google Colab
try:
    import google.colab

    DATA_PATH = COLAB_PATH / Path('data')
    MODELS_PATH = COLAB_PATH / Path('models')
except ImportError:
    # Step 2: Check if running in Kaggle
    try:
        import kaggle_secrets

        DATA_PATH = KAGGLE_PATH
        MODELS_PATH = KAGGLE_PATH
    except ImportError:
        # Step 3: Default to local Jupyter Notebook
        DATA_PATH = LOCAL_PATH / Path('data')
        MODELS_PATH = LOCAL_PATH / Path('models')

GLOVE_6B_PATH = MODELS_PATH / Path('glove-embeddings')
THREAT_TWEETS_PATH = DATA_PATH / Path('tweets-dataset-for-cyberattack-detection')

GLOVE_6B_300D_TXT = GLOVE_6B_PATH / Path('glove.6B.300d.txt')
THREAT_TWEETS_CSV = THREAT_TWEETS_PATH / Path('tweets_final.csv')


## 1. Introduction

In this notebook, we are going to solve a multi-class classification problem using different machine learning models. Our goal is to predict the class of each sample based on the input features.


## 2. Data Loading and Preprocessing
We will load the dataset, inspect its structure, and preprocess it for machine learning models.


In [4]:
# Read the CSV file and process columns in one step
threat_tweets = (
    pd.read_csv(filepath_or_buffer=THREAT_TWEETS_CSV)
    .assign(
        tweet=lambda df: df['tweet'].apply(func=ast.literal_eval),
        watson=lambda df: df['watson'].apply(func=ast.literal_eval)
        .apply(func=lambda x: x.get('categories', []))
        .apply(func=build_tree),
        watson_list=lambda df: df['watson'].apply(func=extract_keys),
    )
    .query(expr='relevant == True')
    .drop(labels=['relevant'], axis=1)
    .dropna(subset=['text'], ignore_index=True)
)

threat_tweets.head()


,_id,date,id,text,tweet,type,watson,annotation,urls,destination_url,valid_certificate,watson_list
0,b'5b8876f9bb325e65fa7e78e4',2018-08-30 23:00:08+00:00,1035301167952211969,Protect your customers access Prestashop Ant...,{'created_at': 'Thu Aug 30 23:00:08 +0000 2018...,ddos,{'technology and computing': {'internet techno...,threat,['http://addons.prestashop.com/en/23513-anti-d...,https://addons.prestashop.com/en/23513-anti-dd...,True,"[technology and computing, internet technology..."
1,b'5b8876f9bb325e65fa7e78e5',2018-08-30 23:00:09+00:00,1035301173178249217,Data leak from Huazhu Hotels may affect 130 mi...,{'created_at': 'Thu Aug 30 23:00:09 +0000 2018...,leak,"{'travel': {'hotels': {}}, 'home and garden': ...",threat,['http://www.hotelmanagement.net/tech/data-lea...,http://www.hotelmanagement.net/tech/data-leak-...,True,"[travel, hotels, home and garden, home improve..."
2,b'5b8876fabb325e65fa7e78e6',2018-08-30 23:00:09+00:00,1035301174583353344,Instagram App 41.1788.50991.0 #Denial Of #Serv...,{'created_at': 'Thu Aug 30 23:00:09 +0000 2018...,general,{'science': {'weather': {'meteorological disas...,threat,['https://packetstormsecurity.com/files/149120...,https://packetstormsecurity.com/files/149120/i...,True,"[science, weather, meteorological disaster, hu..."
3,b'5b88770abb325e65fa7e78e7',2018-08-30 23:00:25+00:00,1035301242271096832,(good slides): \n\nThe Advanced Exploitation o...,{'created_at': 'Thu Aug 30 23:00:25 +0000 2018...,vulnerability,{'business and industrial': {'business operati...,threat,['https://twitter.com/i/web/status/10353012422...,https://twitter.com/i/web/status/1035301242271...,True,"[business and industrial, business operations,..."
4,b'5b887713bb325e65fa7e78e8',2018-08-30 23:00:35+00:00,1035301282095853569,CVE-2018-1000532 (beep)\nhttps://t.co/CaKbo38U...,{'created_at': 'Thu Aug 30 23:00:35 +0000 2018...,vulnerability,{'technology and computing': {'computer securi...,threat,['https://web.nvd.nist.gov/view/vuln/detail?vu...,https://nvd.nist.gov/vuln/detail/CVE-2018-1000532,True,"[technology and computing, computer security, ..."


For the goal of the project, the categories of interest are:
1. computer security/network security
2. computer security/antivirus and malware
3. operating systems/mac os
4. operating systems/windows
5. operating systems/unix
6. operating systems/linux
7. software
8. programming languages, included in software
9. software/databases
10. hardware
11. electronic components, included in hardware
12. hardware/computer/servers
13. hardware/computer/portable computer
14. hardware/computer/desktop computer
15. hardware/computer components
16. hardware/computer networking/router
17. hardware/computer networking/wireless technology
18. networking
19. internet technology, included in networking


In [5]:
FIX_TARGETS = {
    'computer security': 'computer security',
    'operating systems': 'operating systems',
    'software': 'software',
    'programming languages': 'software',
    'hardware': 'hardware',
    'electronic components': 'hardware',
    'networking': 'networking',
    'internet technology': 'networking'
}

chosen_categories = [
    list(set(FIX_TARGETS.keys()) & set(s))
    for s in threat_tweets['watson_list']
]

for i, watson_list in enumerate(chosen_categories):
    temp = list(set([FIX_TARGETS[c] for c in watson_list]))
    if len(temp) < 1:
        temp = ['other']
    chosen_categories[i] = temp

threat_tweets['target'] = chosen_categories

threat_tweets.head()


,_id,date,id,text,tweet,type,watson,annotation,urls,destination_url,valid_certificate,watson_list,target
0,b'5b8876f9bb325e65fa7e78e4',2018-08-30 23:00:08+00:00,1035301167952211969,Protect your customers access Prestashop Ant...,{'created_at': 'Thu Aug 30 23:00:08 +0000 2018...,ddos,{'technology and computing': {'internet techno...,threat,['http://addons.prestashop.com/en/23513-anti-d...,https://addons.prestashop.com/en/23513-anti-dd...,True,"[technology and computing, internet technology...","[networking, software, computer security]"
1,b'5b8876f9bb325e65fa7e78e5',2018-08-30 23:00:09+00:00,1035301173178249217,Data leak from Huazhu Hotels may affect 130 mi...,{'created_at': 'Thu Aug 30 23:00:09 +0000 2018...,leak,"{'travel': {'hotels': {}}, 'home and garden': ...",threat,['http://www.hotelmanagement.net/tech/data-lea...,http://www.hotelmanagement.net/tech/data-leak-...,True,"[travel, hotels, home and garden, home improve...",[other]
2,b'5b8876fabb325e65fa7e78e6',2018-08-30 23:00:09+00:00,1035301174583353344,Instagram App 41.1788.50991.0 #Denial Of #Serv...,{'created_at': 'Thu Aug 30 23:00:09 +0000 2018...,general,{'science': {'weather': {'meteorological disas...,threat,['https://packetstormsecurity.com/files/149120...,https://packetstormsecurity.com/files/149120/i...,True,"[science, weather, meteorological disaster, hu...",[hardware]
3,b'5b88770abb325e65fa7e78e7',2018-08-30 23:00:25+00:00,1035301242271096832,(good slides): \n\nThe Advanced Exploitation o...,{'created_at': 'Thu Aug 30 23:00:25 +0000 2018...,vulnerability,{'business and industrial': {'business operati...,threat,['https://twitter.com/i/web/status/10353012422...,https://twitter.com/i/web/status/1035301242271...,True,"[business and industrial, business operations,...",[operating systems]
4,b'5b887713bb325e65fa7e78e8',2018-08-30 23:00:35+00:00,1035301282095853569,CVE-2018-1000532 (beep)\nhttps://t.co/CaKbo38U...,{'created_at': 'Thu Aug 30 23:00:35 +0000 2018...,vulnerability,{'technology and computing': {'computer securi...,threat,['https://web.nvd.nist.gov/view/vuln/detail?vu...,https://nvd.nist.gov/vuln/detail/CVE-2018-1000532,True,"[technology and computing, computer security, ...","[software, computer security, hardware]"


In [6]:
FIX_TARGETS_COMPUTER_NETWORKING = {
    'router': 'router',
    'wireless technology': 'wireless technology'
}

threat_tweets_computer_networking = threat_tweets[
    threat_tweets["watson_list"].apply(lambda x: "computer networking" in x)
].reset_index(drop=True)

chosen_categories = [
    list(set(FIX_TARGETS_COMPUTER_NETWORKING.keys()) & set(s))
    for s in threat_tweets_computer_networking['watson_list']
]

for i, watson_list in enumerate(chosen_categories):
    temp = list(set([FIX_TARGETS_COMPUTER_NETWORKING[c] for c in watson_list]))

    if len(temp) < 1:
        temp = ['other']

    chosen_categories[i] = temp

print(len(chosen_categories))
threat_tweets_computer_networking['target'] = chosen_categories

threat_tweets_computer_networking.head()


284


,_id,date,id,text,tweet,type,watson,annotation,urls,destination_url,valid_certificate,watson_list,target
0,b'5b8876fabb325e65fa7e78e6',2018-08-30 23:00:09+00:00,1035301174583353344,Instagram App 41.1788.50991.0 #Denial Of #Serv...,{'created_at': 'Thu Aug 30 23:00:09 +0000 2018...,general,{'science': {'weather': {'meteorological disas...,threat,['https://packetstormsecurity.com/files/149120...,https://packetstormsecurity.com/files/149120/i...,True,"[science, weather, meteorological disaster, hu...",[router]
1,b'5b887c5abb325e65fa7e7905',2018-08-30 23:23:05+00:00,1035306945152860160,RCE affecting a huge number of HP printers (lo...,{'created_at': 'Thu Aug 30 23:23:05 +0000 2018...,general,{'technology and computing': {'hardware': {'co...,threat,['https://support.hp.com/us-en/document/c06097...,https://support.hp.com/us-en/document/c06097712,True,"[technology and computing, hardware, computer ...",[router]
2,b'5b888186bb325e65fa7e7939',2018-08-30 23:45:09+00:00,1035312500558913536,Immunity Debugger 1.85 #Denial Of #Service htt...,{'created_at': 'Thu Aug 30 23:45:09 +0000 2018...,general,{'science': {'weather': {'meteorological disas...,threat,['https://packetstormsecurity.com/files/149164...,https://packetstormsecurity.com/files/149164/i...,True,"[science, weather, meteorological disaster, hu...",[router]
3,b'5b888186bb325e65fa7e793a',2018-08-30 23:45:10+00:00,1035312502265892864,Fathom 2.4 #Denial Of #Service https://t.co/cx...,{'created_at': 'Thu Aug 30 23:45:10 +0000 2018...,general,{'science': {'weather': {'meteorological disas...,threat,['https://packetstormsecurity.com/files/149166...,https://packetstormsecurity.com/files/149166/f...,True,"[science, weather, meteorological disaster, hu...",[router]
4,b'5b888507bb325e65fa7e794e',2018-08-31 00:00:07+00:00,1035316263067766784,ipPulse 1.92 TCP Port Denial Of Service (Packe...,{'created_at': 'Fri Aug 31 00:00:07 +0000 2018...,general,{'technology and computing': {'computer securi...,threat,['https://packetstormsecurity.com/files/149165...,https://packetstormsecurity.com/files/149165/i...,True,"[technology and computing, computer security, ...",[router]


In [7]:
Counter([item[0] for item in chosen_categories])


Counter({'router': 270, 'wireless technology': 14})

In [8]:
Counter(tuple(item) for item in chosen_categories)


Counter({('router',): 270, ('wireless technology',): 14})

In [9]:
X = preprocess_texts(
    list_str=threat_tweets_computer_networking['text'],
    model_path=GLOVE_6B_300D_TXT,
    embedding_dim=300
)


## 4. Model Training

We will now train different models and evaluate their performance.


In [10]:
br = None
clr = None
cc = None
lp = None
pst = None
cdn = None
mbr = None
PATH_BR_COMPUTER_NETWORKING = Path('models/binary_problems/br_computer_networking.pkl')
PATH_CLR_COMPUTER_NETWORKING = Path('models/binary_problems/clr_computer_networking.pkl')
PATH_CC_COMPUTER_NETWORKING = Path('models/binary_problems/cc_computer_networking.pkl')
PATH_LP_COMPUTER_NETWORKING = Path('models/multiclass_problems/lp_computer_networking.pkl')
PATH_PST_COMPUTER_NETWORKING = Path('models/multiclass_problems/pst_computer_networking.pkl')
PATH_CDN_COMPUTER_NETWORKING = Path('models/ensembles/cdn_computer_networking.pkl')
PATH_MBR_COMPUTER_NETWORKING = Path('models/ensembles/mbr_computer_networking.pkl')

if PATH_BR_COMPUTER_NETWORKING.exists():
    br = joblib.load(PATH_BR_COMPUTER_NETWORKING)

if PATH_CLR_COMPUTER_NETWORKING.exists():
    clr = joblib.load(PATH_CLR_COMPUTER_NETWORKING)

if PATH_CC_COMPUTER_NETWORKING.exists():
    cc = joblib.load(PATH_CC_COMPUTER_NETWORKING)

if PATH_LP_COMPUTER_NETWORKING.exists():
    lp = joblib.load(PATH_LP_COMPUTER_NETWORKING)

if PATH_PST_COMPUTER_NETWORKING.exists():
    pst = joblib.load(PATH_PST_COMPUTER_NETWORKING)

if PATH_CDN_COMPUTER_NETWORKING.exists():
    cdn = joblib.load(PATH_CDN_COMPUTER_NETWORKING)

if PATH_MBR_COMPUTER_NETWORKING.exists():
    mbr = joblib.load(PATH_MBR_COMPUTER_NETWORKING)


In [11]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y=threat_tweets_computer_networking['target'])

unique_label_sets, threat_tweets_computer_networking['target_lp'] = np.unique(
    ar=y,
    axis=0,
    return_inverse=True
)

X_pst, y_pst, label_map_pst, _ = prune_and_subsample(
    x=X,
    y=y,
    pruning_threshold=10,
    max_sub_samples=30
)

y_lp = threat_tweets_computer_networking['target_lp']

label_map_lp = {i: tuple(lbl_set) for i, lbl_set in enumerate(unique_label_sets)}


In [12]:
# BR, CDR, CC, CDN, MBR
X_train_val, y_train_val, X_test, y_test = iterative_train_test_split(
    X=X,
    y=y,
    test_size=TEST_SIZE
)

X_train, y_train, X_val, y_val = iterative_train_test_split(
    X=X_train_val,
    y=y_train_val,
    test_size=TEST_SIZE
)

# LP
X_train_val_lp, X_test_lp, y_train_val_lp, y_test_lp = train_test_split(
    X, y_lp,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=y_lp
)

X_train_lp, X_val_lp, y_train_lp, y_val_lp = train_test_split(
    X_train_val_lp, y_train_val_lp,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=y_train_val_lp
)

# PSt
X_train_val_pst, X_test_pst, y_train_val_pst, y_test_pst = train_test_split(
    X_pst, y_pst,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=y_pst
)

X_train_pst, X_val_pst, y_train_pst, y_val_pst = train_test_split(
    X_train_val_pst, y_train_val_pst,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=y_train_val_pst
)


### 4.1. Binary Problems


#### 4.1.1. BR (Binary Relevance)


In [13]:
if not br or OVERWRITE:
    br = {}

    for k in tqdm(BASE_CLASSIFIERS.keys()):
        br[k] = OneVsRestClassifier(estimator=BASE_CLASSIFIERS[k]).fit(
            X=X_train,
            y=y_train
        )

    joblib.dump(br, PATH_BR_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:02<00:00,  1.68it/s]


#### 4.1.2. CLR (Calibrated Label Ranking)


In [14]:
if not clr or OVERWRITE:
    clr = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = CalibratedLabelRankClassifier(
            classifier=v,
            classes=mlb.classes_,
            random_state=RANDOM_STATE
        )

        clr[k] = model.fit(
            x=X_train,
            y=[list(mlb.classes_[np.where(row == 1)[0]]) for row in y_train]
        )

    joblib.dump(clr, PATH_CLR_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:09<00:00,  1.91s/it]


#### 4.1.3. CC (Classifier Chains)


In [15]:
if not cc or OVERWRITE:
    cc = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = ChainOfClassifiers(
            classifier=v,
            classes=mlb.classes_,
            random_state=RANDOM_STATE
        )

        cc[k] = model.fit(
            x=X_train,
            y=y_train
        )

    joblib.dump(cc, PATH_CC_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:04<00:00,  1.11it/s]


### 4.2. Multi-class Problems



#### 4.2.1. LP (Label Powerset)


In [16]:
if not lp or OVERWRITE:
    lp = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = LabelPowersetClassifier(
            classifier=v,
            label_map=label_map_lp,
            random_state=RANDOM_STATE
        )

        lp[k] = model.fit(
            x=X_train_lp,
            y=y_train_lp
        )

    joblib.dump(lp, PATH_LP_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:02<00:00,  2.31it/s]


#### 4.2.2. PSt (Pruned Sets)


In [17]:
if not pst or OVERWRITE:
    pst = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = LabelPowersetClassifier(
            classifier=v,
            label_map=label_map_pst,
            random_state=RANDOM_STATE
        )

        pst[k] = model.fit(
            x=X_train_pst,
            y=y_train_pst
        )

    joblib.dump(pst, PATH_PST_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:03<00:00,  1.41it/s]


### 4.3. Ensembles


#### 4.3.1. CDN (Conditional Dependency Network)


In [18]:
if not cdn or OVERWRITE:
    cdn = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = ConditionalDependencyNetwork(
            classifier=v,
            num_iterations=100,
            burn_in=10
        )

        cdn[k] = model.fit(
            x=X_train,
            y=y_train
        )

    joblib.dump(cdn, PATH_CDN_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:04<00:00,  1.05it/s]


#### 4.3.2. MBR (Meta-Binary Relevance)


In [19]:
if not mbr or OVERWRITE:
    mbr = {}

    for k, v in tqdm(BASE_CLASSIFIERS.items()):
        model = MetaBinaryRelevance(
            classifier=v,
            use_cross_val=True,
            n_splits=5
        )

        mbr[k] = model.fit(
            x=X_train,
            y=y_train
        )

    joblib.dump(mbr, PATH_MBR_COMPUTER_NETWORKING, compress=9)


100%|██████████| 5/5 [00:24<00:00,  4.83s/it]


## 5. Model Evaluation

Now that we've trained the models, let's evaluate them in more detail.


In [20]:
evaluation = {
    'BR': assess_models(
        x=X_val,
        y=y_val,
        technique=br
    ),
    'CLR': assess_models(
        x=X_val,
        y=y_val,
        technique=clr
    ),
    'CC': assess_models(
        x=X_val,
        y=y_val,
        technique=cc
    ),
    'LP': assess_models(
        x=X_val_lp,
        y=np.array([list(label_map_lp[yp]) for yp in y_val_lp]),
        technique=lp
    ),
    'PST': assess_models(
        x=X_val_pst,
        y=np.array([list(label_map_pst[yp]) for yp in y_val_pst]),
        technique=pst
    ),
    'CDN': assess_models(
        x=X_val,
        y=y_val,
        technique=cdn
    ),
    'MBR': assess_models(
        x=X_val,
        y=y_val,
        technique=mbr
    )
}


In [21]:
performances = pd.DataFrame(evaluation).T
performances


,Accuracy,Classifier,Model,Precision example-based,Recall example-based,F1 example-based,Hamming loss,Micro precision,Micro recall,Micro F1,Macro precision,Macro recall,Macro F1,Coverage
BR,0.955556,logistic_regression,OneVsRestClassifier(estimator=LogisticRegressi...,0.955556,0.955556,0.955556,0.044444,0.955556,0.955556,0.955556,0.477778,0.5,0.488636,1.044444
CLR,0.955556,random_forest,CalibratedLabelRankClassifier(classes=array(['...,0.955556,0.955556,0.955556,0.044444,0.955556,0.955556,0.955556,0.477778,0.5,0.488636,1.044444
CC,0.955556,random_forest,"ChainOfClassifiers(classes=array(['router', 'w...",0.955556,0.955556,0.955556,0.044444,0.955556,0.955556,0.955556,0.477778,0.5,0.488636,1.044444
LP,0.978261,logistic_regression,LabelPowersetClassifier(classifier=LogisticReg...,0.978261,0.978261,0.978261,0.021739,0.978261,0.978261,0.978261,0.988889,0.75,0.827715,1.021739
PST,0.956522,logistic_regression,LabelPowersetClassifier(classifier=LogisticReg...,0.956522,0.956522,0.956522,0.043478,0.956522,0.956522,0.956522,0.478261,0.5,0.488889,1.043478
CDN,0.933333,logistic_regression,ConditionalDependencyNetwork(classifier=Logist...,0.944444,0.955556,0.948148,0.055556,0.934783,0.955556,0.945055,0.477778,0.5,0.488636,1.066667
MBR,0.955556,logistic_regression,MetaBinaryRelevance(classifier=LogisticRegress...,0.955556,0.955556,0.955556,0.044444,0.955556,0.955556,0.955556,0.477778,0.5,0.488636,1.044444


In [24]:
z = evaluation['LP']['Model'].predict(X_val)

print(classification_report(
    y_true=y_val,
    y_pred=z,
    target_names=mlb.classes_,
    zero_division=0
))


                     precision    recall  f1-score   support

             router       0.98      1.00      0.99        43
wireless technology       1.00      0.50      0.67         2

          micro avg       0.98      0.98      0.98        45
          macro avg       0.99      0.75      0.83        45
       weighted avg       0.98      0.98      0.97        45
        samples avg       0.98      0.98      0.98        45

